In [42]:
# 필요한 라이브러리 임포트
import datetime
import pandas as pd
import FinanceDataReader as fdr
import pmdarima as pm
import plotly.graph_objects as go

# 1. 회사명/종목코드로 데이터 로드
company_name = "삼성전자"  # 또는 종목코드 "005930"
stock_code = "005930"

# 2. 날짜 설정 (최근 1년 + 알파)
end_date = datetime.datetime.now()
start_date = end_date - pd.Timedelta(days=800)  # 여유있게 400일

# 3. 주가 데이터 다운로드
price_df = fdr.DataReader(stock_code, start_date.strftime("%Y%m%d"), end_date.strftime("%Y%m%d"))
# print(f"전체 데이터: {len(price_df)}개")
# price_df.tail()


# 4. 최근 1년치만 학습용으로 추출
close_ser = price_df['Close'].copy()
close_ser.index = pd.to_datetime(close_ser.index)

train_end = close_ser.index.max()
train_start = train_end - pd.Timedelta(days=700)
train = close_ser.loc[train_start:train_end]

# print(f"학습 구간: {train.index.min().date()} ~ {train.index.max().date()} ({len(train)}개)")
# train.tail()

# 5. Auto ARIMA 모델 학습
model = pm.auto_arima(
    train,
    seasonal=False,
    stepwise=True,
    suppress_warnings=True,
    error_action="ignore"
)
# print(model.summary())


# 6. 1달(22영업일) 예측
horizon = 14
forecast = model.predict(n_periods=horizon)

# 7. 예측 결과 데이터프레임
future_idx = pd.bdate_range(train_end + pd.Timedelta(days=1), periods=horizon)
fc_df = pd.DataFrame({"예측종가": forecast.values.astype(int)}, index=future_idx)
# print(fc_df)

# 8. 시각화 (최근 1달 + 예측 7일만)
recent_1month = train.tail(22)  # 최근 약 1개월(영업일 기준)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=recent_1month.index, 
    y=recent_1month, 
    mode="lines", 
    name="종가(최근 1달)",
    line=dict(color='blue')
))
fig.add_trace(go.Scatter(
    x=fc_df.index, 
    y=fc_df["예측종가"], 
    mode="lines", 
    name="예측 종가",
    line=dict(color='red', dash='dot')
))
fig.update_layout(
    title=f"{company_name} 종가 예측",
    # xaxis_title="날짜",
    yaxis_title="종가(원)"
)
fig.show()

/Users/giri/Desktop/ITStudy/04_streamlit_advance/st/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

/Users/giri/Desktop/ITStudy/04_streamlit_advance/st/lib/python3.11/site-packages/statsmodels/tsa/base/tsa_model.py:837: FutureWarning:

No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.

